# 配布された追加のデータを合わせるファイル
# Jリーグ観客動員数を予測せよ！
- Jリーグ公式戦2014年シーズン後半全試合の観客動員数を予測するコンペに挑戦して頂きます。
- コンペページは[こちら](https://signate.jp/courses/VNeoDl9Mpg6YkBQL)

### ライブラリのインポート

In [41]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams["font.family"] = "IPAexGothic" # 日本語の文字化け防止

### データの読み込み
- 基本データは下記の5ファイルです

In [42]:
train = pd.read_csv("data/received/train.csv")
test = pd.read_csv("data/received/test.csv")
condition = pd.read_csv("data/received/condition.csv")
stadium = pd.read_csv("data/received/stadium.csv")
sample = pd.read_csv("data/received/sample_submit.csv",header=None)

### 追加データを結合する
#### 縦結合
##### train
train(train.csv) + train_add(train_add.csv) -> train_new  
train.csvは所々歯抜けになっている為、train_add.csvをtrainに結合する必要がある  
こちらの結合については、**train.csvとtrain_add.csvのカラムは全く等しい**為、単純に縦結合をする  
縦結合をする為には、pd.concat関数を用いる  
例えば、AとBを縦に結合したい場合は、`pd.concat([A, B],sort=False)`と記述する  

In [43]:
# csvの読み込み
train_add = pd.read_csv("data/received/train_add.csv")
# 行数の確認
train_add.shape

(232, 11)

In [44]:
train.shape

(1721, 11)

In [45]:
# 縦に結合
train_new = pd.concat([train,train_add],sort=False)
train_new.shape

(1953, 11)

In [46]:
train_new.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌


##### condition
condition(condition.csv) + condition_add(condition_add.csv) -> condition_new  
condition.csvとcondition_add.csvも縦結合し、新たに変数condition_newを作る

In [47]:
condition_add = pd.read_csv("data/received/condition_add.csv")

In [48]:
condition_add.shape

(270, 31)

In [49]:
condition.shape

(2034, 31)

In [50]:
condition_new = pd.concat([condition,condition_add],sort=False)

In [51]:
condition_new.shape

(2304, 31)

#### 同一キーを元に結合する

##### trainとconditionとstadiumを結合する -> train_allを作成

1. train_newとcondition_newを結合し、変数tmpを作成  
2. tmpとstadiumを結合し、変数train_allを作成

〜作り方〜  
今度は単純な結合ではなく、同一キーを元に結合することになる為、pd.merge関数を使う  
具体的には、AとBをカラムXにて結合したい場合は、`pd.merge(A,B,on="X")`と記述する  
結合のヒントとなるカラムが複数の場合は、`pd.merge(A,B,on=["X,Y"])`と記述する  
AとBとで、結合のヒントとなるカラム名が異なる場合は、`pd.merge(A,B,left_on="X",right_on="Y")`とする  
結合時に、左のデータフレームに寄せたい場合は、`pd.merge(A,B,how="left")`と記述する  

1.  idで横に結合 (train_new + condition_new) -> tmp 

In [52]:
train_new.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌


In [53]:
condition_new.head()

,id,home_score,away_score,weather,temperature,humidity,referee,home_team,home_01,home_02,...,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,13994,1,0,雨,3.8,66%,木村 博之,ベガルタ仙台,林 卓人,菅井 直樹,...,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ
1,13995,1,0,屋内,12.4,43%,西村 雄一,名古屋グランパス,楢﨑 正剛,田中 隼磨,...,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀
2,13996,2,3,晴一時雨,11.3,41%,高山 啓義,ガンバ大阪,藤ヶ谷 陽介,加地 亮,...,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人
3,13997,1,0,曇一時雨のち晴,11.4,52%,松尾 一,サンフレッチェ広島,西川 周作,森脇 良太,...,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也
4,13998,0,0,屋内,22.5,32%,廣瀬 格,コンサドーレ札幌,李 昊乗,高木 純平,...,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一


In [54]:
train_new.shape

(1953, 11)

In [55]:
condition_new.shape

(2304, 31)

In [56]:
tmp = pd.merge(train_new, condition_new, on="id")
tmp.shape

(1953, 41)

In [57]:
tmp.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,...,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,...,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,...,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,...,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,...,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,...,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一


In [58]:
tmp.isnull().sum()

id             0
y              0
year           0
stage          0
match          0
gameday        0
time           0
home           0
away           0
stadium        0
tv             0
home_score     0
away_score     0
weather        0
temperature    0
humidity       0
referee        0
home_team      0
home_01        0
home_02        0
home_03        0
home_04        0
home_05        0
home_06        0
home_07        0
home_08        0
home_09        0
home_10        0
home_11        0
away_team      0
away_01        0
away_02        0
away_03        0
away_04        0
away_05        0
away_06        0
away_07        0
away_08        0
away_09        0
away_10        0
away_11        0
dtype: int64

1. stadiumで参照し横に結合 (tmp + stadium) -> train_all

In [59]:
tmp.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,...,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,...,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ
1,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,...,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀
2,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,...,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人
3,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,...,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也
4,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,...,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一


In [60]:
stadium.head()

,name,address,capa
0,名古屋市瑞穂陸上競技場,愛知県名古屋市瑞穂区山下通5-1,20000
1,豊田スタジアム,愛知県豊田市千石町7-2,40000
2,フクダ電子アリーナ,千葉県千葉市中央区川崎町1-20,18500
3,日立柏サッカー場,千葉県柏市日立台1-2-50,15349
4,ニンジニアスタジアム,愛媛県松山市上野町乙46,15576


In [61]:
tmp.shape

(1953, 41)

In [62]:
train_all = pd.merge(tmp,stadium,left_on="stadium",right_on="name")
train_all = train_all.sort_values(by="id")
train_all.shape

(1953, 44)

In [63]:
train_all.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,...,away_05,away_06,away_07,away_08,away_09,away_10,away_11,name,address,capa
0,13994,18250,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,...,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
41,13995,24316,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,...,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀,豊田スタジアム,愛知県豊田市千石町7-2,40000
66,13996,17066,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,...,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人,万博記念競技場,大阪府吹田市千里万博公園5-2,21000
112,13997,29603,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,...,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也,エディオンスタジアム広島,広島県広島市安佐南区大塚西5-1-1,50000
155,13998,25353,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,...,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一,札幌ドーム,北海道札幌市豊平区羊ヶ丘1,39232


In [64]:
train_all.isnull().sum()

id             0
y              0
year           0
stage          0
match          0
gameday        0
time           0
home           0
away           0
stadium        0
tv             0
home_score     0
away_score     0
weather        0
temperature    0
humidity       0
referee        0
home_team      0
home_01        0
home_02        0
home_03        0
home_04        0
home_05        0
home_06        0
home_07        0
home_08        0
home_09        0
home_10        0
home_11        0
away_team      0
away_01        0
away_02        0
away_03        0
away_04        0
away_05        0
away_06        0
away_07        0
away_08        0
away_09        0
away_10        0
away_11        0
name           0
address        0
capa           0
dtype: int64

In [65]:
#スタジアムの名前が"stadium","name"の両方にあり紛らわしい為、片方(name)を削除
train_all = train_all.drop(columns="name")
#文字コードをutf8に統一する為に、encoding="utf8"というオプションを記載
train_all.to_csv("data/create/train_all.csv",index=None,encoding="utf8")

#### testも同じ処理をする
(trainデータとtestデータは同じ形でいなければいけない)
1. testとcondition_newの行数・列数を確認しよう
1. testとcondition_newをmergeし、変数tmpに代入しよう
1. tmpの行数・列数及び欠損がないかを確認しよう
1. tmpとstadiumを結合し、変数test_allに代入しよう
    - 結合後、idでソートをしよう
1. test_allの行数・列数及び欠損がないかを確認しよう
1. test_allをファイル名「test_all.csv」として書き出そう
    - 保存前に重複しているカラムnameは削除しよう
    - index=Noneとencoding="utf8"を忘れずに

In [66]:
test.shape

(313, 10)

In [67]:
condition_new.shape

(2304, 31)

In [68]:
test.head()

,id,year,stage,match,gameday,time,home,away,stadium,tv
0,15822,2014,Ｊ１,第１８節第１日,08/02(土),19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス
1,15823,2014,Ｊ１,第１８節第１日,08/02(土),18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス
2,15824,2014,Ｊ１,第１８節第１日,08/02(土),19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉
3,15825,2014,Ｊ１,第１８節第１日,08/02(土),19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス
4,15827,2014,Ｊ１,第１８節第１日,08/02(土),19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス


In [69]:
condition_new.head()

,id,home_score,away_score,weather,temperature,humidity,referee,home_team,home_01,home_02,...,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,13994,1,0,雨,3.8,66%,木村 博之,ベガルタ仙台,林 卓人,菅井 直樹,...,新井場 徹,岩政 大樹,中田 浩二,アレックス,青木 剛,増田 誓志,小笠原 満男,本山 雅志,大迫 勇也,ジュニーニョ
1,13995,1,0,屋内,12.4,43%,西村 雄一,名古屋グランパス,楢﨑 正剛,田中 隼磨,...,吉田 豊,岩下 敬輔,カルフィン ヨン ア ピン,李 記帝,村松 大輔,河井 陽介,枝村 匠馬,高木 俊幸,アレックス,大前 元紀
2,13996,2,3,晴一時雨,11.3,41%,高山 啓義,ガンバ大阪,藤ヶ谷 陽介,加地 亮,...,近藤 岳登,北本 久仁衛,伊野波 雅彦,相馬 崇人,三原 雅俊,田中 英雄,野沢 拓也,橋本 英郎,森岡 亮太,大久保 嘉人
3,13997,1,0,曇一時雨のち晴,11.4,52%,松尾 一,サンフレッチェ広島,西川 周作,森脇 良太,...,濱田 水輝,阿部 勇樹,槙野 智章,平川 忠亮,鈴木 啓太,山田 直輝,梅崎 司,柏木 陽介,原口 元気,田中 達也
4,13998,0,0,屋内,22.5,32%,廣瀬 格,コンサドーレ札幌,李 昊乗,高木 純平,...,駒野 友一,チョ ビョングク,藤田 義明,山本 脩斗,小林 裕紀,山本 康裕,山田 大記,松浦 拓弥,菅沼 実,前田 遼一


In [70]:
tmp = pd.merge(test,condition_new,on="id")
tmp.head()

,id,year,stage,match,gameday,time,home,away,stadium,tv,...,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,15822,2014,Ｊ１,第１８節第１日,08/02(土),19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,...,中村 北斗,今井 智基,横山 知伸,高橋 祥平,橋本 晃司,和田 拓也,家長 昭博,富山 貴光,ムルジャ,ズラタン
1,15823,2014,Ｊ１,第１８節第１日,08/02(土),18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス,...,塩谷 司,千葉 和彦,水本 裕貴,柏 好文,柴﨑 晃誠,森﨑 和幸,山岸 智,石原 直樹,髙萩 洋次郎,佐藤 寿人
2,15824,2014,Ｊ１,第１８節第１日,08/02(土),19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉,...,河本 裕之,増川 隆洋,大屋 翼,橋本 英郎,杉浦 恭平,奥井 諒,高橋 峻希,小川 慶治朗,森岡 亮太,マルキーニョス
3,15825,2014,Ｊ１,第１８節第１日,08/02(土),19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス,...,小宮山 尊信,實藤 友紀,谷口 彰悟,登里 享平,大島 僚太,中村 憲剛,森谷 賢太郎,金久保 順,小林 悠,大久保 嘉人
4,15827,2014,Ｊ１,第１８節第１日,08/02(土),19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス,...,藤本 康太,丸橋 祐介,安藤 淳,山下 達也,扇原 貴宏,山口 蛍,キム ソンジュン,フォルラン,南野 拓実,杉本 健勇


In [71]:
tmp.shape

(313, 40)

In [72]:
tmp.isnull().sum()

id             0
year           0
stage          0
match          0
gameday        0
time           0
home           0
away           0
stadium        0
tv             0
home_score     0
away_score     0
weather        0
temperature    0
humidity       0
referee        0
home_team      0
home_01        0
home_02        0
home_03        0
home_04        0
home_05        0
home_06        0
home_07        0
home_08        0
home_09        0
home_10        0
home_11        0
away_team      0
away_01        0
away_02        0
away_03        0
away_04        0
away_05        0
away_06        0
away_07        0
away_08        0
away_09        0
away_10        0
away_11        0
dtype: int64

In [73]:
stadium.head()

,name,address,capa
0,名古屋市瑞穂陸上競技場,愛知県名古屋市瑞穂区山下通5-1,20000
1,豊田スタジアム,愛知県豊田市千石町7-2,40000
2,フクダ電子アリーナ,千葉県千葉市中央区川崎町1-20,18500
3,日立柏サッカー場,千葉県柏市日立台1-2-50,15349
4,ニンジニアスタジアム,愛媛県松山市上野町乙46,15576


In [74]:
tmp = pd.merge(tmp,stadium,left_on="stadium",right_on="name")
tmp.head()

,id,year,stage,match,gameday,time,home,away,stadium,tv,...,away_05,away_06,away_07,away_08,away_09,away_10,away_11,name,address,capa
0,15822,2014,Ｊ１,第１８節第１日,08/02(土),19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,...,高橋 祥平,橋本 晃司,和田 拓也,家長 昭博,富山 貴光,ムルジャ,ズラタン,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
1,15840,2014,Ｊ１,第２０節第１日,08/16(土),19:04,ベガルタ仙台,清水エスパルス,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,...,イ キジェ,本田 拓也,六平 光成,大前 元紀,河井 陽介,高木 俊幸,ノヴァコヴィッチ,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
2,15863,2014,Ｊ１,第２２節第１日,08/30(土),19:04,ベガルタ仙台,横浜Ｆ・マリノス,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,...,下平 匠,中町 公祐,小椋 祥平,藤本 淳吾,中村 俊輔,兵藤 慎剛,伊藤 翔,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
3,15885,2014,Ｊ１,第２５節第１日,09/23(火・祝),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,...,山本 脩斗,柴崎 岳,小笠原 満男,遠藤 康,カイオ,土居 聖真,ダヴィ,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
4,15903,2014,Ｊ１,第２７節第１日,10/05(日),15:34,ベガルタ仙台,ＦＣ東京,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス／東日本放送,...,太田 宏介,高橋 秀人,米本 拓司,羽生 直剛,河野 広貴,エドゥー,武藤 嘉紀,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694


In [75]:
test_all = tmp.sort_values(by="id")
test_all

,id,year,stage,match,gameday,time,home,away,stadium,tv,...,away_05,away_06,away_07,away_08,away_09,away_10,away_11,name,address,capa
0,15822,2014,Ｊ１,第１８節第１日,08/02(土),19:04,ベガルタ仙台,大宮アルディージャ,ユアテックスタジアム仙台,スカパー！／スカパー！プレミアムサービス,...,高橋 祥平,橋本 晃司,和田 拓也,家長 昭博,富山 貴光,ムルジャ,ズラタン,ユアテックスタジアム仙台,宮城県仙台市泉区七北田字柳78,19694
8,15823,2014,Ｊ１,第１８節第１日,08/02(土),18:34,鹿島アントラーズ,サンフレッチェ広島,県立カシマサッカースタジアム,スカパー！／スカパー！プレミアムサービス,...,柏 好文,柴﨑 晃誠,森﨑 和幸,山岸 智,石原 直樹,髙萩 洋次郎,佐藤 寿人,県立カシマサッカースタジアム,茨城県鹿嶋市神向寺後山26-2,40728
17,15824,2014,Ｊ１,第１８節第１日,08/02(土),19:04,浦和レッズ,ヴィッセル神戸,埼玉スタジアム２００２,スカパー！／スカパー！プレミアムサービス／ＮＨＫ ＢＳ１／テレ玉,...,橋本 英郎,杉浦 恭平,奥井 諒,高橋 峻希,小川 慶治朗,森岡 亮太,マルキーニョス,埼玉スタジアム２００２,埼玉県さいたま市緑区中野田500,63700
25,15825,2014,Ｊ１,第１８節第１日,08/02(土),19:03,柏レイソル,川崎フロンターレ,日立柏サッカー場,スカパー！／スカパー！プレミアムサービス,...,登里 享平,大島 僚太,中村 憲剛,森谷 賢太郎,金久保 順,小林 悠,大久保 嘉人,日立柏サッカー場,千葉県柏市日立台1-2-50,15349
33,15827,2014,Ｊ１,第１８節第１日,08/02(土),19:03,アルビレックス新潟,セレッソ大阪,デンカビッグスワンスタジアム,スカパー！／スカパー！プレミアムサービス,...,山下 達也,扇原 貴宏,山口 蛍,キム ソンジュン,フォルラン,南野 拓実,杉本 健勇,デンカビッグスワンスタジアム,新潟県新潟市中央区清五郎67-12,42300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,16432,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:04,カターレ富山,ファジアーノ岡山,富山県総合運動公園陸上競技場,スカパー！／スカパー！プレミアムサービス,...,田中 奏一,島田 譲,上田 康太,三村 真,片山 瑛一,押谷 祐樹,清水 慎太郎,富山県総合運動公園陸上競技場,富山県富山市南中田368,25250
205,16433,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:04,京都サンガF.C.,ＦＣ岐阜,京都市西京極総合運動公園陸上競技場兼球技場,スカパー！／スカパー！プレミアムサービス／ＮＨＫ京都／ＮＨＫ岐阜／ＫＢＳ京都（録）,...,森 勇介,ヘニキ,宮沢 正史,比嘉 諒人,髙地 系治,難波 宏明,ナザリト,京都市西京極総合運動公園陸上競技場兼球技場,京都府京都市右京区西京極新明町29,20588
236,16434,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:03,カマタマーレ讃岐,ジェフユナイテッド千葉,香川県立丸亀競技場,スカパー！／スカパー！プレミアムサービス,...,中村 太亮,佐藤 勇人,佐藤 健太郎,幸野 志有人,町田 也真人,谷澤 達也,森本 貴幸,香川県立丸亀競技場,香川県丸亀市金倉町830,22338
245,16435,2014,Ｊ２,第４２節第１日,11/23(日・祝),14:04,アビスパ福岡,ロアッソ熊本,レベルファイブスタジアム,スカパー！／スカパー！プレミアムサービス,...,園田 拓也,嶋田 慎太郎,養父 雄仁,髙柳 一誠,澤田 崇,齊藤 和樹,アンデルソン,レベルファイブスタジアム,福岡県福岡市博多区東平尾公園2-1-1,22563


In [76]:
test_all.shape

(313, 43)

In [77]:
test_all.isnull().sum()

id             0
year           0
stage          0
match          0
gameday        0
time           0
home           0
away           0
stadium        0
tv             0
home_score     0
away_score     0
weather        0
temperature    0
humidity       0
referee        0
home_team      0
home_01        0
home_02        0
home_03        0
home_04        0
home_05        0
home_06        0
home_07        0
home_08        0
home_09        0
home_10        0
home_11        0
away_team      0
away_01        0
away_02        0
away_03        0
away_04        0
away_05        0
away_06        0
away_07        0
away_08        0
away_09        0
away_10        0
away_11        0
name           0
address        0
capa           0
dtype: int64

In [78]:
#スタジアムの名前が"stadium","name"の両方にあり紛らわしい為、片方(name)を削除
test_all = test_all.drop(columns="name")
#文字コードをutf8に統一する為に、encoding="utf8"というオプションを記載
test_all.to_csv("data/create/test_all.csv",index=None,encoding="utf8")